TensorFlow and Keras

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow import keras

2025-11-28 18:49:11.949763: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-28 18:49:11.993589: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-28 18:49:13.023581: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [3]:
from keras.preprocessing.image import load_img

In [4]:
path = 'clothing-dataset-small-master/train/t-shirt'
name = '5f0a3fa0-6a3d-4b68-b213-72766a643de7.jpg'
fullname = f'{path}/{name}'


In [5]:
img = load_img(fullname, target_size=(299, 299))

In [6]:
x = np.array(img)
x.shape

(299, 299, 3)

Pre-trained convolutional neural networks

In [7]:
from keras.applications.xception import Xception, preprocess_input, decode_predictions

In [8]:
model = Xception(weights='imagenet', input_shape=(299, 299, 3))

E0000 00:00:1764355757.587858   55653 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1764355757.603603   55653 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
X = np.array([x])
X.shape

(1, 299, 299, 3)

In [10]:
X = preprocess_input(X)

In [11]:
pred = model.predict(X)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 585ms/step


In [12]:
decode_predictions(pred)

[[('n03595614', 'jersey', np.float32(0.68196285)),
  ('n02916936', 'bulletproof_vest', np.float32(0.038140062)),
  ('n04370456', 'sweatshirt', np.float32(0.034324847)),
  ('n03710637', 'maillot', np.float32(0.011354239)),
  ('n04525038', 'velvet', np.float32(0.0018453649))]]

Transfer learning

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [1]:
# Build image generator for training (takes preprocessing input function)
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load in train dataset into train generator
train_ds = train_gen.flow_from_directory(directory='clothing-dataset-small-master/train', # Train images directory
                                         target_size=(150,150), # resize images to train faster
                                         batch_size=32) # 32 images per batch

NameError: name 'ImageDataGenerator' is not defined

In [ ]:
# Create image generator for validation
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Load in image for validation
val_ds = val_gen.flow_from_directory('clothing-dataset-small-master/validation', # Validation image directory
                                     target_size=(150,150),
                                     batch_size=32,
                                     shuffle=False) # False for validation

In [ ]:
train_ds.class_indices

In [ ]:
base_model = Xception(
    weights='imagenet', 
    include_top=False, 
    input_shape=(150, 150, 3)
)
base_model.trainable = False

Create new top

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))

base = base_model(inputs)

outputs = base

model = keras.Model(inputs, outputs)

preds = model.predict(X)

In [ ]:
preds.shape

In [ ]:
X.shape